In [1]:
# Load modules necessary
import os
import numpy as np
import matplotlib.pyplot as plt
import mne
import pandas as pd
from scipy.signal import detrend

%matplotlib qt

Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.


## Load sample dataset

##### For EEG dataset, we will use CHB-MIT Scalp EEG Dataset from here: https://physionet.org/content/chbmit/1.0.0/chb01/#files-panel

##### The original dataset contains 22 subjects. However, here in this tutorial we will explore data from Subject 01
##### You can download the dataset from Brightspace

## Vectors (norm, dot product, cosine rule)

##### A vector is a way of representing a data point in N-dimensional space. For example,
##### v = [1, 2] is a 2-dimensional vector with x = 1 and y = 2
##### w = [1, 2, 3] is a 3-dimensional vector with x = 1, y = 2, z = 3
##### A vector has both length and direction

##### Norm of a vector captures the length of the vector

##### Cosine rule: 
##### cos($\theta$) = $\frac{(x * y)}{||x||.||y||}$

In [ ]:
v1 = np.array([1, 2, 3])
v2 = np.array([3, 4, 5])
v3 = np.array([3, 6, 9])



In [2]:
# Path to the EEG file
eegPath = '../../Datasets/EEG/sub-01/eeg/sub-01_task-daf_eeg_filtered.vhdr'

# Load the EEG file using MNE
# MNE has different read formats for different EEG file types
# Here we are using read_raw_edf to read the EEG file
# preload=True loads the data into memory (default is False, which loads the data when needed)
raw = mne.io.read_raw_brainvision(eegPath, preload=True)
elecPos = pd.read_csv('../../Datasets/iEEG/sub-01/eeg/sub-01_electrodes.tsv', sep='\t')
# Add fiducials
fiducials = pd.DataFrame({
    'name': ['Nz', 'LPA', 'RPA'],
    'x': [-4.129838157917329e-18, -0.0729282673627754, 0.08278152042487033],
    'y': [0.10011015398430487, 3.008505424862354e-18, -3.414981080487009e-18],
    'z': [-5.7777898331617076e-33, 3.851859888774472e-34, 3.4666738998970245e-33]
})

# Concatenate the original electrode positions with the fiducials
elecPos = pd.concat([elecPos, fiducials], ignore_index=True)

montage = mne.channels.make_dig_montage(
    ch_pos=dict(zip(elecPos['name'], elecPos[['x', 'y', 'z']].values)),
    coord_frame='head'
)
raw.set_montage(montage)

Extracting parameters from ../../Datasets/EEG/sub-01/eeg/sub-01_task-daf_eeg_filtered.vhdr...
Setting channel info structure...
Reading 0 ... 244237  =      0.000 ...   976.948 secs...


<RawBrainVision | sub-01_task-daf_eeg_filtered.eeg, 66 x 244238 (976.9 s), ~123.1 MB, data loaded>

In [3]:
# Creating a fake channel with a polynomial trend
fake_data = raw.get_data()[0]
t = np.linspace(0, 1, len(fake_data))

np.random.seed(42)
trend = np.random.randn(3)
fake_data += 4e-3 * np.polyval(trend, t)

# Let us look at different ways to remove trend from the data
# First, we can simply mean-center the data
# This is done by subtracting the mean of the data from the data
fake_data_centered = fake_data - np.mean(fake_data)

# Second, we can remove a linear trend from the data
# This can be done using detrend() function of numpy
fake_data_detrended = detrend(fake_data, type='linear')

# Next we can remove a polynomial trend from the data
# This is done by fitting a polynomial model to the data and subtracting the model from the data
# We can use the polyfit() function of numpy to fit a polynomial model to the data
# The polyfit() function returns the coefficients of the polynomial model
# We can use the polyval() function of numpy to evaluate the polynomial model at the data points
# We can then subtract the polynomial model from the data
trend = np.polyfit(t, fake_data, 3)
fake_data_detrended2 = fake_data - np.polyval(trend, t)

# Plot the original data and the detrended data
f, axs = plt.subplots(2, 2, figsize=(10, 10))
axs[0, 0].plot(t, fake_data)
axs[0, 0].set_title('Original Data')
axs[0, 1].plot(t, fake_data_centered)
axs[0, 1].set_title('Mean-Centered Data')
axs[1, 0].plot(t, fake_data_detrended)
axs[1, 0].set_title('Linear Detrended Data')
axs[1, 1].plot(t, fake_data_detrended2)
axs[1, 1].set_title('Polynomial Detrended Data')
plt.show()